# Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras_preprocessing.image import ImageDataGenerator
import utils
import shutil

# Parameters

Using max pooling instead of average pooling could lead to better results as it could help keep the edges crisp and make the leaves pop out from the background, preserving their shape better.

In [2]:
data_dir = '../input/flowerspeciesclassification/images'
model_file_name = 'densenet121'
pooling = 'max'  # vs 'avg'
batch_size = 128

In [3]:
RNG_SEED: int = 7
VAL_SPLIT: float = 0.2

# Augmented Data Generator parameters
# Transformations
ROTATION_RANGE: float = 20. # Max rotation in degrees
ZOOM_RANGE: float = 0.15 # Max zoom
WIDTH_SHIFT_RANGE: int = 10 # Horizontal shift (in pixels)
HEIGHT_SHIFT_RANGE: int = 10 # Vertical shift (in pixels)
HORIZONTAL_FLIP: bool = True # Horizontal flip
VERTICAL_FLIP: bool = False # Vertical flip
BRIGHTNESS_SHIFT_RANGE: list = [0.8, 1.2] # Minimum and max brightness scaling
SHEAR_RANGE: float = None # Max shear in degrees

FILL_MODE: str = "reflect" # Filling method for out-of-border pixels

# Create the augmented image data pipe

In [4]:
aug_data_gen = ImageDataGenerator(
    rotation_range=ROTATION_RANGE,  # Max rotation in degrees
    zoom_range=ZOOM_RANGE,  # Max zoom
    width_shift_range=WIDTH_SHIFT_RANGE,  # Horizontal shift (in pixels)
    height_shift_range=HEIGHT_SHIFT_RANGE,  # Vertical shift (in pixels)
    horizontal_flip=HORIZONTAL_FLIP,  # Horizontal flip
    vertical_flip=VERTICAL_FLIP,  # Vertical flip
    brightness_range=BRIGHTNESS_SHIFT_RANGE,  # Minimum and max brightness scaling
    shear_range=SHEAR_RANGE,  # Max shear in degrees
    fill_mode=FILL_MODE,  # Filling method for out-of-border pixels
    validation_split=VAL_SPLIT
)

Train-validation split using ImageDataGenerator object

In [5]:
aug_train_data = aug_data_gen.flow_from_directory(
    directory=data_dir,
    target_size=(96,96),
    batch_size=batch_size,
    subset="training"
)

aug_val_data = aug_data_gen.flow_from_directory(
    directory=data_dir,
    target_size=(96,96),
    batch_size=batch_size,
    subset="validation",
    shuffle=False
)

Found 2836 images belonging to 8 classes.
Found 706 images belonging to 8 classes.


# Build the model

In [6]:
# Import the base CNN model, keep only the feature-extraction, convolutional part
base_model = tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(96, 96, 3),
    pooling=pooling,
    classes=8,
)
# Freeze the base CNN model
for layer in base_model.layers:
    layer.trainable = False

inputs = keras.Input(shape=(96, 96, 3))

x = keras.applications.densenet.preprocess_input(inputs)
x = base_model(x, training=T)  # set training = False to be in inference mode
                                    # i.e not recompute batch norm

# --- Feature interpretation / fully-connected part ---
# We add 2 intermediate layers, half the size of the feature vector extracted by the CNN
# We add dropout as a mean of regularization
x = keras.layers.Dense(x.shape[-1]//2, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Dense(x.shape[-1], activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)

# Set initial biases for classes based on their distribution in the given dataset (not necessary)
initial_biases = keras.initializers.Constant(utils.initial_class_biases)

# Connect to the output layer with softmax activation
outputs = keras.layers.Dense(8, activation="softmax",
                            bias_initializer=initial_biases)(x)

model = keras.Model(inputs, outputs)
model.summary()

2022-11-16 21:59:34.588933: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 21:59:34.672045: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 21:59:34.672800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 21:59:34.673923: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

29097984/29084464 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 96, 96, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 96, 96, 3)         0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 96, 96, 3)         0         
_________________________________________________________________
densenet121 (Functional)     (None, 1024)              7037504   
_________________________________________________________________
dense (Dense)                (None, 512)               524800    
______________________________________________________________

# Training the fully-connected layers (the feature interpretation part)

Let's compile the resulting model with Adam as optimizer function and categorical cross entropy. We make sure to keep track of the accuracy as this is the metric we want to improve for the classification task.

In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics='accuracy'
)

In [8]:
# We give weights for each class to contribute diffrently to the loss function,
# such that missed detections on more rare classes will have more impact on the loss function
class_weight = {spec: weight for (spec, weight) in zip(range(utils.n_species), utils.class_loss_weights)}

model.fit(
    aug_train_data,
    batch_size=batch_size,
    epochs=200,
    validation_data=aug_val_data,
    class_weight=class_weight,
    callbacks = [
        keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=15, restore_best_weights=True),
        keras.callbacks.TensorBoard(
            log_dir="tensorboard_logs", 
            profile_batch=0,
            histogram_freq=1
        )  # if > 0 (epochs) shows weights histograms
    ]
)

2022-11-16 21:59:41.866942: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200


2022-11-16 21:59:48.530074: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


23/23 [==============================] - 26s 628ms/step - loss: 4.0322 - accuracy: 0.3635 - val_loss: 1.7549 - val_accuracy: 0.5453
Epoch 2/200
23/23 [==============================] - 14s 589ms/step - loss: 1.8397 - accuracy: 0.5226 - val_loss: 1.2864 - val_accuracy: 0.5878
Epoch 3/200
23/23 [==============================] - 13s 553ms/step - loss: 1.5442 - accuracy: 0.5716 - val_loss: 1.4643 - val_accuracy: 0.5623
Epoch 4/200
23/23 [==============================] - 13s 573ms/step - loss: 1.4373 - accuracy: 0.5864 - val_loss: 1.4716 - val_accuracy: 0.5793
Epoch 5/200
23/23 [==============================] - 13s 547ms/step - loss: 1.5254 - accuracy: 0.5815 - val_loss: 1.6736 - val_accuracy: 0.5057
Epoch 6/200
23/23 [==============================] - 13s 564ms/step - loss: 1.4449 - accuracy: 0.5913 - val_loss: 1.2871 - val_accuracy: 0.6218
Epoch 7/200
23/23 [==============================] - 13s 555ms/step - loss: 1.4263 - accuracy: 0.5927 - val_loss: 1.5594 - val_accuracy: 0.5708
Epoc

In [9]:
model.save(model_file_name + "_frozen")

2022-11-16 22:13:35.467008: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [10]:
shutil.make_archive(model_file_name + "_frozen", 'zip', model_file_name + "_frozen")

'/kaggle/working/densenet121_noupscaling_frozen.zip'

# Fine-tuning the full model including feature-extraction

Unfreeze the convolutive model and recompile the total model. We will train again using a much lower learning rate in order to fine-tune the weights in the feature-extraction part of the model, i.e the `DenseNet121`.

In [11]:
# Unfreeze the base CNN model
for layer in base_model.layers:
    layer.trainable = True

model.compile(
    optimizer=keras.optimizers.Adam(1e-4),  # or even 1e-6, low learning rate is necessary for fine-tuning
    loss=keras.losses.CategoricalCrossentropy(),
    metrics='accuracy'
)

model.fit(
    aug_train_data,
    batch_size=batch_size,
    epochs=200,
    validation_data=aug_val_data,
    class_weight=class_weight,
    callbacks = [
        keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True),
        keras.callbacks.TensorBoard(
            log_dir="tensorboard_logs", 
            profile_batch=0,
            histogram_freq=1
        )  # if > 0 (epochs) shows weights histograms
    ]
)

Epoch 1/200
23/23 [==============================] - 31s 729ms/step - loss: 1.5961 - accuracy: 0.4905 - val_loss: 1.0984 - val_accuracy: 0.5680
Epoch 2/200
23/23 [==============================] - 14s 614ms/step - loss: 0.9960 - accuracy: 0.6492 - val_loss: 0.9315 - val_accuracy: 0.6388
Epoch 3/200
23/23 [==============================] - 14s 621ms/step - loss: 0.8330 - accuracy: 0.6999 - val_loss: 0.7997 - val_accuracy: 0.7252
Epoch 4/200
23/23 [==============================] - 14s 617ms/step - loss: 0.7262 - accuracy: 0.7320 - val_loss: 0.8358 - val_accuracy: 0.6912
Epoch 5/200
23/23 [==============================] - 15s 622ms/step - loss: 0.6452 - accuracy: 0.7669 - val_loss: 0.6951 - val_accuracy: 0.7394
Epoch 6/200
23/23 [==============================] - 14s 627ms/step - loss: 0.6021 - accuracy: 0.7898 - val_loss: 0.8279 - val_accuracy: 0.6898
Epoch 7/200
23/23 [==============================] - 14s 609ms/step - loss: 0.5791 - accuracy: 0.7898 - val_loss: 0.6760 - val_accuracy:

In [12]:
model.save(model_file_name)

In [13]:
shutil.make_archive(model_file_name, 'zip', model_file_name)

'/kaggle/working/densenet121_noupscaling.zip'

In [14]:
shutil.make_archive("tensorboard_logs_" + model_file_name, 'zip', "tensorboard_logs")

'/kaggle/working/tensorboard_logs_densenet121_noupscaling.zip'